## Predição de valor de aposentadoria

Partindo de um servidor público que recebe abono permanência, deseja-se estimar seu ganho quando se aposentar

In [78]:
import pandas as pd

df = pd.read_csv("../servidores.csv")

Como os nomes não interessam ao modelo, podemos retirá-lo da tabela

In [79]:
df = df.drop('nome', axis=1)
df = df.dropna()

Convertendo a coluna de escolaridade em uma coluna numérica

In [80]:
discrete_esco = []

for esco in df.escolaridade:
    if esco == "SEGUNDO GRAU INCOMPLETO" or esco == "4A. SERIE DO PRIMEIRO GRAU COMPLETA":
        discrete_esco.append(0)
    elif esco == "ALFABETIZADO SEM CURSOS REGULARES" or esco == "ENSINO FUNDAMENTAL INCOMPLETO":
        discrete_esco.append(1)
    elif esco == "ENSINO FUNDAMENTAL" or esco == "ENSINO MEDIO":
        discrete_esco.append(2)
    elif esco == "SUPERIOR INCOMPLETO" or esco == "ENSINO SUPERIOR":
        discrete_esco.append(3)
    else:
        discrete_esco.append(4)

df['escolaridade'] = discrete_esco

In [81]:
# Ordenando os estados pela quantidade de ocorrências
ordered_estates = []
key = df.UF_residencia.value_counts().index
for i in range(27):
    ordered_estates.append(key[i])

ordered_estates = ordered_estates[::-1]

discrete_uf = []
for uf in df.UF_residencia:
    for i in range(27):
        if ordered_estates[i] == uf:
            discrete_uf.append(i)

df.UF_residencia = discrete_uf

Preparando os dados de treinamento e teste

In [85]:
from sklearn.model_selection import train_test_split

X = df.drop('valor_aposentadoria', axis=1)
y = df.valor_aposentadoria


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=11)

realizando o treinamento utilizando um modelo de RandomForest para regressão

In [86]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor()

In [88]:
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate

# Testando os modelos contra os dados de teste
y_pred = model.predict(X_test)

# F1 Score
#f1_score = f1_score(y_test, y_pred, average='macro')

#print(f"F1 Score: {f1_score}\n")

# K-fold cross-validation
k = 10
results = cross_validate(model, X_train,y_train, cv = k)

print(f"10-fold Score: {results['test_score'].mean()}")

10-fold Score: 0.13669317466190764
